In [24]:
!pip install catboost

In [25]:
import pickle
import numpy as np
import pandas as pd
import joblib
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [26]:
X_test = pd.read_csv('X_test_full.csv')
y_test = pd.read_csv('y_test_full.csv')

In [27]:
model1 = CatBoostClassifier().load_model('catboost.pkl')

In [28]:
model2 = joblib.load('random_forest_classifier.pkl')

In [29]:
accuracy1 = 0.9270
accuracy2 = 0.9307
total = accuracy1 + accuracy2
weight1 = accuracy1 / total
weight2 = accuracy2 / total

In [30]:
class WeightedEnsemble:
    def __init__(self, model1, model2, weight1, weight2):
        self.model1 = model1
        self.model2 = model2
        self.weight1 = weight1
        self.weight2 = weight2

    def predict_proba(self, X):
        proba1 = self.model1.predict_proba(X)[:, 1]
        proba2 = self.model2.predict_proba(X)[:, 1]
        ensemble_proba = self.weight1 * proba1 + self.weight2 * proba2
        return np.vstack((1 - ensemble_proba, ensemble_proba)).T

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)


In [31]:
ensemble_model = WeightedEnsemble(model1, model2, weight1, weight2)

ensemble_pred = ensemble_model.predict(X_test)
print("Ensemble Classification Report:\n", classification_report(y_test, ensemble_pred, digits=4))
print("Confusion Matrix:\n", confusion_matrix(y_test, ensemble_pred))
print("Accuracy:", accuracy_score(y_test, ensemble_pred))

Ensemble Classification Report:
               precision    recall  f1-score   support

           0     0.9922    0.9221    0.9559    401878
           1     0.9273    0.9928    0.9589    401946

    accuracy                         0.9575    803824
   macro avg     0.9598    0.9575    0.9574    803824
weighted avg     0.9598    0.9575    0.9574    803824

Confusion Matrix:
 [[370574  31304]
 [  2895 399051]]
Accuracy: 0.9574546169310695


In [32]:
joblib.dump(ensemble_model, 'ensemble_model.pkl')


['ensemble_model.pkl']